In [2]:
import polars as pl
from src.config import Settings
from src.db_tools import DBEngine

# Create connection
db_engine = DBEngine(**Settings().model_dump())

In [4]:
with db_engine.session.connection() as con:
    df = pl.read_database(
        query="SELECT * FROM regios",
        connection=con
    )

In [7]:
df

regio_key,regio,description,categorygroupid
str,str,str,bool
"""PV21 ""","""Fryslân (PV)""","""PV = Provincie…",null
"""GM0305""","""Abcoude""","""Opgeheven per …",null
"""GM0059""","""Achtkarspelen""","""Grenswijziging…",null
"""GM0531""","""Hendrik-Ido-Am…","""Grenswijziging…",null
"""LD01 ""","""Noord-Nederlan…","""LD = Landsdeel…",null
"""LD04 ""","""Zuid-Nederland…","""LD = Landsdeel…",null
"""GM0248""","""Hengelo (Gld.)…","""Opgeheven per …",null
"""GM0881""","""Onderbanken""","""Opgeheven per …",null
"""PV23 ""","""Overijssel (PV…","""PV = Provincie…",null
